You might need to install this on your system:

apt-get install python3-opencv git

In [1]:
import os
#"""
# ! rm -r k
if not os.path.isdir('k'):
 !git clone -b development9 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
#"""
!cd k && pip install .

Already up to date.
Processing /tf/k
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=43485 sha256=e6eb2f32b64ebbd1d931b2c6208abda5400c52d286e0800b48269fc9696cbaa3
  Stored in directory: /tmp/pip-ephem-wheel-cache-yk1eg4za/wheels/f0/08/19/56f64e8c8cc45b0390e5e7e2f634c4c1aa0212065044fb6442
Successfully built cai
  Attempting uninstall: cai
    Found existing installation: cai 0.1.4
    Uninstalling cai-0.1.4:
      Successfully uninstalled cai-0.1.4
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import cai.layers
import cai.datasets
import cai.models
import cai.inception_v3
import numpy as np
from tensorflow import keras
from tensorflow.keras import mixed_precision
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
from sklearn.metrics import classification_report

Tensorflow version: 2.6.0
Keras version: 2.6.0


In [3]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [4]:
# mixed_precision.set_global_policy('mixed_float16')

# Create folder structure with CIFAR-10 as png files.

In [5]:
verbose=True
root_folder = 'cifar10-as-png'
data_dir = root_folder + '/train';
test_dir = root_folder + '/test';

x_train, y_train, x_test, y_test = cai.datasets.load_dataset(tf.keras.datasets.cifar10, verbose=verbose, lab=False, bipolar=False)

if not os.path.isdir(root_folder):
  os.mkdir(root_folder)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_train)*255, y_train, dest_folder_name=data_dir)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_test)*255, y_test, dest_folder_name=test_dir)

170508288/170498071 [==============================] - 26s 0us/step
train shape (50000, 32, 32, 3)
test shape (10000, 32, 32, 3)
Original channel  0  min: 0.0  max: 255.0
Original channel  1  min: 0.0  max: 255.0
Original channel  2  min: 0.0  max: 255.0
Loading RGB.
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0


In [6]:
num_classes = 10
batch_size = 64
epochs = 50
target_size_x = 224
target_size_y = 224
seed = 12

In [7]:
train_datagen = cai.util.create_image_generator(validation_split=0.1, rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.1)
test_datagen = cai.util.create_image_generator_no_augmentation() # preprocessing_function=cai.util.rgb2bipolar_lab
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [8]:
def work_on_kinception_v3(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [-1, 2, 3]:
      basefilename = 'DEV-kInceptionV3-CIFAR10-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.inception_v3.InceptionV3(
          include_top=True,
          weights=None,
          input_tensor=None,
          input_shape=input_shape,
          pooling=None,
          classes=num_classes)
      else:
        model = cai.inception_v3.two_path_inception_v3(
          include_top=True,
          weights=None, #'two_paths_plant_leafs'
          input_shape=input_shape,
          pooling=None,
          classes=num_classes,
          two_paths_partial_first_block=0,
          two_paths_first_block=False,
          two_paths_second_block=False,
          deep_two_paths=False,
          deep_two_paths_compression=0.5,
          deep_two_paths_bottleneck_compression=0.5,
          l_ratio=0.5,
          ab_ratio=0.5,
          # max_mix_idx=5,
          max_mix_deep_two_paths_idx=-1,
          model_name='two_path_inception_v3',
          kType=kType)
        
      #opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model):
        model.summary()
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        train_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="training",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        validation_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="validation",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        history = model.fit(
          x = train_flow,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_flow,
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=cpus_num,
          max_queue_size=128
          )
      if (test_results):
        test_flow = test_datagen.flow_from_directory(
            directory=test_dir,
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(x_test)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(y_test, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [9]:
work_on_kinception_v3(show_model=True, run_fit=False, test_results=False)

Running: DEV-kInceptionV3-CIFAR10--1
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
__________________________________________________

# Fitting

In [10]:
work_on_kinception_v3(show_model=False, run_fit=True, test_results=True)

Running: DEV-kInceptionV3-CIFAR10--1
Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Epoch 1/50
704/704 [==============================] - 674s 904ms/step - loss: 1.9678 - accuracy: 0.2682 - val_loss: 2.0398 - val_accuracy: 0.2606

Epoch 00001: val_accuracy improved from -inf to 0.26060, saving model to DEV-kInceptionV3-CIFAR10--1-best_result.hdf5
Epoch 2/50
704/704 [==============================] - 628s 884ms/step - loss: 1.6326 - accuracy: 0.4020 - val_loss: 2.3863 - val_accuracy: 0.3860

Epoch 00002: val_accuracy improved from 0.26060 to 0.38600, saving model to DEV-kInceptionV3-CIFAR10--1-best_result.hdf5
Epoch 3/50
704/704 [==============================] - 619s 867ms/step - loss: 1.4410 - accuracy: 0.4808 - val_loss: 8.9121 - val_accuracy: 0.3492

Epoch 00003: val_accuracy did not improve from 0.38600
Epoch 4/50
704/704 [==============================] - 614s 865ms/step - loss: 1.2670 - accuracy: 0.5464 - val_loss: 2.4168 - val_accuracy: 0.

/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
704/704 [==============================] - 412s 581ms/step - loss: 1.4215 - accuracy: 0.4982 - val_loss: 1.6469 - val_accuracy: 0.4524

Epoch 00002: val_accuracy improved from 0.30740 to 0.45240, saving model to DEV-kInceptionV3-CIFAR10-2-best_result.hdf5
Epoch 3/50
704/704 [==============================] - 418s 586ms/step - loss: 1.2258 - accuracy: 0.5720 - val_loss: 8.4585 - val_accuracy: 0.3836

Epoch 00003: val_accuracy did not improve from 0.45240
Epoch 4/50
704/704 [==============================] - 415s 584ms/step - loss: 1.1093 - accuracy: 0.6138 - val_loss: 1.3813 - val_accuracy: 0.5374

Epoch 00004: val_accuracy improved from 0.45240 to 0.53740, saving model to DEV-kInceptionV3-CIFAR10-2-best_result.hdf5
Epoch 5/50
704/704 [==============================] - 413s 582ms/step - loss: 0.9948 - accuracy: 0.6536 - val_loss: 1.6120 - val_accuracy: 0.5420

Epoch 00005: val_accuracy improved from 0.53740 to 0.54200, saving model to DEV-kInceptionV3-CIFAR10-2-best_result.hd

# Test Results

In [11]:
work_on_kinception_v3(show_model=False, run_fit=False, test_results=True)

Running: DEV-kInceptionV3-CIFAR10--1
Found 10000 images belonging to 10 classes.
Best Model Results: DEV-kInceptionV3-CIFAR10--1-best_result.hdf5
157/157 [==============================] - 21s 125ms/step - loss: 3.0189 - accuracy: 0.8829
loss 3.018876314163208
acc 0.8828999996185303
Finished: DEV-kInceptionV3-CIFAR10--1
Running: DEV-kInceptionV3-CIFAR10-2
Found 10000 images belonging to 10 classes.
Best Model Results: DEV-kInceptionV3-CIFAR10-2-best_result.hdf5
157/157 [==============================] - 27s 157ms/step - loss: 0.3235 - accuracy: 0.9110
loss 0.323459655046463
acc 0.9110000133514404
Finished: DEV-kInceptionV3-CIFAR10-2
Running: DEV-kInceptionV3-CIFAR10-3
Found 10000 images belonging to 10 classes.
Best Model Results: DEV-kInceptionV3-CIFAR10-3-best_result.hdf5
157/157 [==============================] - 22s 129ms/step - loss: 0.2942 - accuracy: 0.9171
loss 0.2941713333129883
acc 0.9171000123023987
Finished: DEV-kInceptionV3-CIFAR10-3
